# Previsão Presença Deputados AR

### A ter em consideração
<ul>
    <li>Se o feriado calhou a um Fim de Semana, não é relevante saber se faltaram 2ª ou 6ª</li>
    <li>Idealmente podemos criar 3 features relevantes, relativamente às faltas:</li>
    <ol>
        <li>Presença ou falta (1 ou 0) à 2ª/6ª antes/depois de feriado à 3ª/5ª</li>
        <li>Presença ou falta (1 ou 0) antes ou depois de Feriado à 4ª</li>
        <li>Presença ou falta (1 ou 0) à 3ª/5ª depois/antes de feriado à 2ª/6ª</li>
    </ol>
    <li>Para a previsão ser ao nível do deputado, temos que treinar o modelo por deputado, cada vez que quisermos fazer uma previsão. Tentar incorporar esta funcionalidade desde início</li>
     <li>Adicionar previsão do tempo para o feriado</li>
     <li>Greves marcadas?</li>
     <li>Cotação do PSI20 no dia útil anterior à potencial falta</li>
</ul>

In [65]:
import pandas as pd
raw_data = pd.read_csv('presencas_ar.csv', sep=',')
type(raw_data)

pandas.core.frame.DataFrame

In [66]:
raw_data.describe()

,deputado_id,presente,dia_semana,segunda_feira,terca_feira,quarta_feira,quinta_feira,sexta_feira,antes_de_feriado,depois_de_feriado
count,191747.000000,191747.000000,191747.000000,191747.000000,191747.000000,191747.000000,191747.000000,191747.000000,191747.000000,191747.000000
mean,281.836509,0.938064,3.832420,0.020428,0.036157,0.325528,0.321543,0.293945,0.032407,0.032397
std,374.744516,0.241040,0.971527,0.141459,0.186681,0.468573,0.467070,0.455568,0.177080,0.177052
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,82.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,163.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,253.000000,1.000000,5.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,1413.000000,1.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [67]:
raw_data.head()

,deputado_id,data,presente,dia_semana,segunda_feira,terca_feira,quarta_feira,quinta_feira,sexta_feira,antes_de_feriado,depois_de_feriado
0,1,15-10-2009,1,4,0,0,0,1,0,0,0
1,2,15-10-2009,1,4,0,0,0,1,0,0,0
2,3,15-10-2009,1,4,0,0,0,1,0,0,0
3,233,15-10-2009,1,4,0,0,0,1,0,0,0
4,392,15-10-2009,1,4,0,0,0,1,0,0,0


### First Pass
#### In this pass we only drop "id" and "data" and use the data as is

In [68]:
# Experimentar filtrar um deputado_id.
deputado_id_filter = 3

In [69]:
pd_train = raw_data.drop(['data', 'dia_semana'], axis=1)

In [70]:
pd_train.describe()

,deputado_id,presente,segunda_feira,terca_feira,quarta_feira,quinta_feira,sexta_feira,antes_de_feriado,depois_de_feriado
count,191747.000000,191747.000000,191747.000000,191747.000000,191747.000000,191747.000000,191747.000000,191747.000000,191747.000000
mean,281.836509,0.938064,0.020428,0.036157,0.325528,0.321543,0.293945,0.032407,0.032397
std,374.744516,0.241040,0.141459,0.186681,0.468573,0.467070,0.455568,0.177080,0.177052
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,82.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,163.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,253.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,1413.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [71]:
# Filtrar apenas o deputado que nos interesa
pd_train = pd_train.loc[pd_train['deputado_id'] == deputado_id_filter]

In [72]:
# Agora já não nos interessa a coluna "deputado id"
pd_train = pd_train.drop(['deputado_id'], axis=1)
pd_train.head()

,presente,segunda_feira,terca_feira,quarta_feira,quinta_feira,sexta_feira,antes_de_feriado,depois_de_feriado
2,1,0,0,0,1,0,0,0
232,1,0,0,0,1,0,0,0
462,1,0,0,0,0,1,0,0
692,1,0,0,1,0,0,0,0
922,1,0,0,1,0,0,0,0


In [73]:
pd_train.shape

(836, 8)

In [74]:
# Agora temos que transformar o DataFrame em Numpy, aproveitar e separar em test,train
from sklearn.model_selection import train_test_split

# Criar Y_data
y_train_temp = pd_train.filter(items=["presente"]).values
y_train = y_train_temp.flatten()
print('HERE COMES y_data')
print(y_train.shape)

# Criar X_data
X_train = pd_train.filter(items=["segunda_feira", "terca_feira", "quarta_feira", "quinta_feira", "sexta_feira", "antes_de_feriado", "depois_de_feriado"]).values
print('HERE COMES X_data')
print(X_train.shape)

HERE COMES y_data
(836,)
HERE COMES X_data
(836, 7)


In [75]:
# just some parameters we'll use in the forloops in each model to test
# several models at the same time
from random import randint

c_values = [0.01, 0.1, 1, 10]
kernel_types = ['linear', 'poly', 'rbf', 'sigmoid']
activ_nn = ['logistic', 'relu', 'identity']
solver_nn = ['lbfgs', 'sgd', 'adam']
randomstate = randint(0,100)

In [76]:
# now we use the model_selection to split our data in trainning and cross validation
from sklearn.model_selection import train_test_split

X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X_train, y_train, test_size=0.2, random_state=randomstate)

In [77]:
# start trying with linear regression
from sklearn.linear_model import LogisticRegression
import numpy as np

# start a loop to itirate over all the parameters we are trying out
# i change y_train first due to an error that tells me to use gravel on the 
for c in c_values:
    LR_clf = LogisticRegression(C=c, random_state=randomstate)
    LR_clf.fit(X_train_final, y_train_final)
    score = LR_clf.score(X_test_final, y_test_final)
    print("Linear Regression | C = %f | Score = %f"% (c, score))

Linear Regression | C = 0.010000 | Score = 0.839286
Linear Regression | C = 0.100000 | Score = 0.839286
Linear Regression | C = 1.000000 | Score = 0.839286
Linear Regression | C = 10.000000 | Score = 0.839286


In [78]:
# continue our models test with SVMs
from sklearn.svm import SVC

for c in c_values:
    for kern in kernel_types:
        SVM_clf = SVC(C=c, kernel=kern, gamma='auto', random_state=randomstate)
        SVM_clf.fit(X_train_final, y_train_final)
        score = SVM_clf.score(X_test_final, y_test_final)
        print("Support Vector Machines | C = %f | Kernel = %s | Score = %f"% (c, kern, score))

Support Vector Machines | C = 0.010000 | Kernel = linear | Score = 0.839286
Support Vector Machines | C = 0.010000 | Kernel = poly | Score = 0.839286
Support Vector Machines | C = 0.010000 | Kernel = rbf | Score = 0.839286
Support Vector Machines | C = 0.010000 | Kernel = sigmoid | Score = 0.839286
Support Vector Machines | C = 0.100000 | Kernel = linear | Score = 0.839286
Support Vector Machines | C = 0.100000 | Kernel = poly | Score = 0.839286
Support Vector Machines | C = 0.100000 | Kernel = rbf | Score = 0.839286
Support Vector Machines | C = 0.100000 | Kernel = sigmoid | Score = 0.839286
Support Vector Machines | C = 1.000000 | Kernel = linear | Score = 0.839286
Support Vector Machines | C = 1.000000 | Kernel = poly | Score = 0.839286
Support Vector Machines | C = 1.000000 | Kernel = rbf | Score = 0.839286
Support Vector Machines | C = 1.000000 | Kernel = sigmoid | Score = 0.839286
Support Vector Machines | C = 10.000000 | Kernel = linear | Score = 0.839286
Support Vector Machines

In [79]:
# now we try with sklearn neural network
from sklearn.neural_network import MLPClassifier

for activ in activ_nn:
    for solv in solver_nn:
        nn_clf = MLPClassifier(hidden_layer_sizes=(500,400), activation=activ, solver=solv,
                              max_iter=1000)
        nn_clf.fit(X_train_final, y_train_final)
        score = nn_clf.score(X_test_final, y_test_final)
        print("Neural Networks | Activation = %s | Solver = %s | Score = %f" % (activ, solv, score))

Neural Networks | Activation = logistic | Solver = lbfgs | Score = 0.839286
Neural Networks | Activation = logistic | Solver = sgd | Score = 0.839286
Neural Networks | Activation = logistic | Solver = adam | Score = 0.839286
Neural Networks | Activation = relu | Solver = lbfgs | Score = 0.839286
Neural Networks | Activation = relu | Solver = sgd | Score = 0.839286
Neural Networks | Activation = relu | Solver = adam | Score = 0.839286
Neural Networks | Activation = identity | Solver = lbfgs | Score = 0.839286
Neural Networks | Activation = identity | Solver = sgd | Score = 0.839286
Neural Networks | Activation = identity | Solver = adam | Score = 0.839286
